# 결측값 확인

In [1]:
import pandas as pd


def check_missing_values(df):
    missing_counts = df.isnull().sum()
    missing_columns = missing_counts[missing_counts > 0]

    if len(missing_columns) == 0:
        print("결측값이 없습니다.")
        return False
    else:
        print("\n결측값이 있는 열:")
        for column, count in missing_columns.items():
            ratio = (count / len(df)) * 100
            print(f"- {column}: {count}개 (전체 데이터의 {ratio:.2f}%)")
        return True


years = [2020, 2021, 2022, 2023]

for year in years:
    print(f"{year}년 전기차 충전량 데이터 결측값 확인")
    df = pd.read_csv(f"data/hourly_ev_charge_{year}.csv")
    check_missing_values(df)
    print(f"{year}년 기온 데이터 결측값 확인")
    df = pd.read_csv(f"data/hourly_temp_{year}.csv")
    check_missing_values(df[["기온(°C)"]])
    print("- - -")

2020년 전기차 충전량 데이터 결측값 확인
결측값이 없습니다.
2020년 기온 데이터 결측값 확인
결측값이 없습니다.
- - -
2021년 전기차 충전량 데이터 결측값 확인
결측값이 없습니다.
2021년 기온 데이터 결측값 확인
결측값이 없습니다.
- - -
2022년 전기차 충전량 데이터 결측값 확인
결측값이 없습니다.
2022년 기온 데이터 결측값 확인
결측값이 없습니다.
- - -
2023년 전기차 충전량 데이터 결측값 확인
결측값이 없습니다.
2023년 기온 데이터 결측값 확인
결측값이 없습니다.
- - -


# 전기차 충전량 데이터 전처리

In [2]:
from holidayskr import is_holiday
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

for year in years:
    df = pd.read_csv(f"data/hourly_ev_charge_{year}.csv")

    # 시간대별 충전량 합산
    time_columns = [f"{i}시" for i in range(24)]
    df["충전량"] = df[time_columns].sum(axis=1)

    # 일별 충전량 계산
    daily_ev_charge = df.groupby("일자")["충전량"].sum().reset_index()
    daily_ev_charge.rename(columns={"일자": "날짜"}, inplace=True)

    # 날짜 형식으로 변환
    daily_ev_charge["날짜"] = pd.to_datetime(daily_ev_charge["날짜"])

    # 주말과 공휴일 제거
    daily_ev_charge = daily_ev_charge[
        ~daily_ev_charge["날짜"].dt.weekday.isin([5, 6])  # 주말 제거
        & ~daily_ev_charge["날짜"]
        .dt.strftime("%Y-%m-%d")
        .apply(is_holiday)  # 공휴일 제거
        ]

    # 결과 저장
    daily_ev_charge.to_csv(
        f"data/daily_ev_charge_{year}.csv", index=False, encoding="utf-8"
    )

# 기상청 기온 데이터 전처리

In [3]:
for year in years:
    df = pd.read_csv(f"data/hourly_temp_{year}.csv")

    # 날짜/시간 변환
    df["일시"] = pd.to_datetime(df["일시"])
    df["날짜"] = df["일시"].dt.date

    # 일평균 기온 계산
    daily_temp_data = df.groupby("날짜")["기온(°C)"].mean().round(2).reset_index()
    daily_temp_data.rename(columns={"기온(°C)": "기온"}, inplace=True)

    # 날짜를 datetime 형식으로 변환
    daily_temp_data["날짜"] = pd.to_datetime(daily_temp_data["날짜"])

    # 주말과 공휴일 제거
    daily_temp_data = daily_temp_data[
        ~daily_temp_data["날짜"].dt.weekday.isin([5, 6])  # 주말 제거
        & ~daily_temp_data["날짜"]
        .dt.strftime("%Y-%m-%d")
        .apply(is_holiday)  # 공휴일 제거
        ]

    # 결과 저장
    daily_temp_data.to_csv(f"data/daily_temp_{year}.csv", index=False, encoding="utf-8")